In [1]:
from chinese_checkers.simulation import SimulationCatalog, GameSimulation, SimulationMetadata

from typing import List

import pandas as pd
from tqdm import tqdm
from collections import Counter

In [ ]:
# filter to only 2 player games on a size 4 board
catalog = SimulationCatalog()
metadata_list: List[SimulationMetadata] = [
    m
    for m in catalog.list_available_metadata()
    if m.player_count == 2 and m.board_size == 4
]

for meta in metadata_list:
    print(meta)

In [ ]:
simulations: List[GameSimulation] = []

with tqdm(metadata_list, desc="Loading Simulations") as progress_bar:
    for metadata in progress_bar:
        simulations += list(catalog.load_simulations_by_metadata(metadata))
        progress_bar.set_postfix({"Simulation Count": len(simulations)})

print("All simulations loaded.")

In [ ]:
# compute the summary statistics for the simulations
data = []
for sim in simulations:
    metadata: SimulationMetadata = sim.metadata
    data.append({
        "name": metadata.name,
        "version": metadata.version,
        "winning_player": metadata.winning_player,
        "game_length": len(sim.data.historical_moves)
    })

simulations_df = pd.DataFrame(data)

summary_df = simulations_df.groupby(["name", "version", "winning_player"]).agg(
    game_count=("winning_player", "size"),
    avg_game_length=("game_length", "mean"),
    min_game_length=("game_length", "min"),
    max_game_length=("game_length", "max"),
).reset_index()

metadata_summary_df = simulations_df.groupby(["name", "version"]).agg(
    total_games=("winning_player", "size"),
    winning_player_counts=("winning_player", lambda x: Counter(x))
).reset_index()

full_summary_df = pd.merge(summary_df, metadata_summary_df, on=["name", "version"])
full_summary_df

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

all_moves = []
for sim in simulations:
    all_moves.extend(sim.data.historical_moves)

move_counts = Counter(all_moves)

move_df = pd.DataFrame(list(move_counts.items()), columns=["move", "count"])

total_moves = len(all_moves)
unique_moves = move_df[move_df["count"] == 1].shape[0]
duplicate_moves = move_df[move_df["count"] > 1].shape[0]
max_duplicate_count = move_df["count"].max()
mean_duplicate_count = move_df[move_df["count"] > 1]["count"].mean()

summary_stats = {
    "total_moves": total_moves,
    "unique_moves": unique_moves,
    "duplicate_moves": duplicate_moves,
    "max_duplicate_count": max_duplicate_count,
    "mean_duplicate_count": mean_duplicate_count,
}
summary_df = pd.DataFrame([summary_stats])

print(summary_df)

plt.figure(figsize=(10, 6))
move_df[move_df["count"] > 1]["count"].plot(kind="hist", bins=200, edgecolor="black")
plt.title("Distribution of Duplicate Move Counts")
plt.xlabel("Duplicate Count")
plt.ylabel("Frequency")
plt.show()
